In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [40]:
from datasets import load_dataset, Audio, Dataset
import pandas as pd
import librosa
import soundfile
import random

In [36]:
dataset = load_dataset(
    "mayabedge/NNCES", split="train"
)
dataset

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 500
})

In [37]:
sample = dataset[2]
print(sample)
print(sample["transcription"])

{'audio': {'path': 'M1_01_03.wav', 'array': array([ 0.00167847,  0.0015564 ,  0.00140381, ..., -0.00192261,
       -0.0027771 , -0.00332642]), 'sampling_rate': 44100}, 'transcription': 'Mom and Dad arrived at the bus station early but waited until the bus came.'}
Mom and Dad arrived at the bus station early but waited until the bus came.


In [38]:
import torch
from transformers import pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
pipe = pipeline(
    "automatic-speech-recognition", model="openai/whisper-tiny.en", device=device
)

In [41]:
preds = []
evals = random.sample(range(0, 500), 100)
for i in evals:
  sample = dataset[i]
  txt = pipe(sample["audio"], max_new_tokens=256)
  txt = txt['text']
  preds.append(txt)
print(preds)


[' mother and dad arrived at the bus station early but waited until the bus came', " I don't have time to play with you. I am playing quicken.", ' Mom and Dad arrived at the bus station early but waited until the bus came.', ' The oldest member of the family is my grandfather and the youngest member of my family is my baby brother.', ' which is as caring as a mother. We talk with sweet like', ' Ancient views are through the best habits and dreams to eat.', ' I love my family and that in that future I want to make my family proud of.', ' Mom and Dad and I were at the bus station and he went way down until the bus came.', ' We should respect and know all members of the family.', " I don't have time to whoa, it's you, I am playing cricket.", ' The nature is very attractive and full of my favorite ginkalurt.', ' The nature is very attractive and full of my few favorite green colors.', " I don't have time to go with you. I'm playing cricket.", " I don't have time to go with you. I'm playing

In [43]:
refs = []
for i in evals:
  sample = dataset[i]
  refs.append(sample["transcription"])

In [32]:
!pip install jiwer
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00


In [44]:
from evaluate import load
wer = load("wer")
wer_score = wer.compute(predictions=preds, references=refs)
print(f"WER score: {wer_score}")

WER score: 0.43286573146292584


In [35]:
print(refs)
print(preds)

['I wish to know all about my grandfather. Well he is nearly 93 years old with a long beard.', 'In the wenter when it snows, he slowly takes a short walk in the open air each day.', '100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80.', 'Dog eats bones. Mike likes bikes. Elsa wants a book. Adam plays basketball.', 'The students wear the uniform and go to school daily.', "I don't have time to go with you. I am playing cricket.", 'The nature is very attractive and full of my favorite green color.', 'Nature gives us fruits, vegetables, and grains to eat.', "I don't have time to go with you. I am playing cricket.", 'The nature is very attractive and full of my favorite green color.', 'I do not have time to go with you. I am playing cricket.', 'The nature is very attractive and full of my favorite green color.', "Nature is as caring as our mother. It's always sweet like her.", 'You wish to know all about my grandfather? Well he is nearly 93 years old with a 